# Graph Attention Networks

**WARNING**<br>
Execute only the colab

In [ ]:
# install torch-geometric and related libraries
!pip install pyg-lib torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-1.13.0+cu116.html
!pip install torch-geometric
!pip install scipy==1.8.0

## Load the data

In [2]:
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader

# load dataset
dataset = TUDataset(root='data/TUDataset', name='MUTAG')

# split dataset into train, validation and test
dataset = dataset.shuffle()
train_dataset = dataset[:140]
val_dataset = dataset[140:]

# create dataloader
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

Extracting data/TUDataset/MUTAG/MUTAG.zip
Processing...
Done!
/Users/takamatsunaoki/opt/anaconda3/envs/ai-master/lib/python3.7/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


## Build the model

In [ ]:
# define model
# import libraries
import torch
import torch.nn as nn
from torch_geometric.nn import GATConv, global_mean_pool

# set parameters
n_features = dataset.num_features
n_hidden = 64
n_heads = 32
n_classes = dataset.num_classes

# define model class
class GAT(nn.Module):
    def __init__(self):
        super(GAT, self).__init__()
        self.gat1 = GATConv(n_features, n_hidden, heads=n_heads)
        self.gat2 = GATConv(n_hidden * n_heads, 1, heads=n_heads)
        self.fc1 = nn.Linear(n_hidden * n_heads, n_classes)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        batch = data.batch

        x = self.gat1(x, edge_index)
        x = self.relu(x)
        x = self.gat2(x, edge_index)
        x = self.relu(x)
        x = global_mean_pool(x, batch)
        x = self.dropout(x)
        x = self.fc1(x)

        return x
    
net = GAT()
net.cuda()

## Train the model

In [ ]:
# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters())

# define function to evaluate model
def evaluate(loader):
    correct = 0
    total = 0
    for data in loader:
        data = data.cuda()
        outputs = net(data)
        _, predicted = torch.max(outputs.data, 1)
        total += data.y.size(0)
        correct += (predicted == data.y).sum().item()
    return correct / total * 100

# train model
n_epochs = 200
for epoch in range(n_epochs):
    for i, data in enumerate(train_loader):
        data = data.cuda()
        optimizer.zero_grad()
        outputs = net(data)
        loss = criterion(outputs, data.y)
        loss.backward()
        optimizer.step()
    net.eval()
    train_acc = evaluate(train_loader)
    val_acc = evaluate(val_loader)
    print(f'Epoch {epoch+1}/{n_epochs}, Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%')

## Evaluate the model

In [ ]:
# evaluate model
net.eval()
test_acc = evaluate(val_loader)
print(f'Test Acc: {test_acc:.2f}%')